# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [2]:
# Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [5]:
# Your code here
q = """
SELECT lastName, firstName
FROM employees AS e
JOIN offices AS o
ON e.officeCode = o.officeCode
WHERE o.city = "Boston";
"""

pd.read_sql(q, conn)

,lastName,firstName,city
0,Firrelli,Julie,Boston
1,Patterson,Steve,Boston


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [6]:
# Your code here
q = """
SELECT COUNT(*) as num_employees, o.officeCode, o.city
FROM employees as e
JOIN offices as o
ON e.officeCode = o.officeCode
GROUP BY o.officeCode
"""
pd.read_sql(q, conn)

,num_employees,officeCode,city
0,6,1,San Francisco
1,2,2,Boston
2,2,3,NYC
3,5,4,Paris
4,2,5,Tokyo
5,4,6,Sydney
6,2,7,London


## Write 3 questions of your own and answer them

In [ ]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

In [8]:
"""
Question 1
What are the names and # of orders of the 5 most ordered products?
"""

# Your code here
q = """
SELECT p.productName, COUNT(*) as num_orders
FROM products AS p
JOIN orderdetails as o
ON p.productCode = o.productCode
GROUP BY p.productCode
ORDER BY num_orders DESC
LIMIT 5;
"""
pd.read_sql(q, conn)

,productName,num_orders
0,1992 Ferrari 360 Spider red,53
1,Boeing X-32A JSF,28
2,American Airlines: MD-11S,28
3,F/A 18 Hornet 1/72,28
4,ATA: B757-300,28


In [11]:
"""
Question 2
Who are the 10 customers with the greatest # of orders?
"""

# Your code here
q = """
SELECT c.customerName, COUNT(*) as num_orders
FROM customers as c
JOIN orders as o
ON c.customerNumber = o.customerNumber
GROUP BY c.customerName
ORDER BY num_orders DESC
LIMIT 10;
"""
pd.read_sql(q, conn)

,customerName,num_orders
0,Euro+ Shopping Channel,26
1,Mini Gifts Distributors Ltd.,17
2,Reims Collectables,5
3,"Dragon Souveniers, Ltd.",5
4,"Down Under Souveniers, Inc",5
5,Danish Wholesale Imports,5
6,"Australian Collectors, Co.",5
7,"Volvo Model Replicas, Co",4
8,"Tokyo Collectables, Ltd",4
9,The Sharp Gifts Warehouse,4


In [12]:
"""
Question 3
Which 3 Vendors supplied the most orders?
"""

# Your code here
q = """
SELECT p.productVendor, COUNT(*) as total_num_orders
FROM products as p
JOIN orderdetails as o
ON p.productCode = o.productCode
GROUP BY p.productVendor
ORDER BY total_num_orders DESC
LIMIT 3;
"""
pd.read_sql(q, conn)

,productVendor,total_num_orders
0,Classic Metal Creations,270
1,Motor City Art Classics,249
2,Carousel DieCast Legends,246


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [13]:
# Your code here
q = """
SELECT e.lastName, e.firstName, p.productName
FROM employees AS e
JOIN customers AS c
ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders AS o
ON c.customerNumber = o.customerNumber
JOIN orderdetails AS od
ON o.orderNumber = od.orderNumber
JOIN products AS p
ON od.productCode = p.productCode
"""
pd.read_sql(q, conn)

,lastName,firstName,productName
0,Jennings,Leslie,1958 Setra Bus
1,Jennings,Leslie,1940 Ford Pickup Truck
2,Jennings,Leslie,1939 Cadillac Limousine
3,Jennings,Leslie,1996 Peterbilt 379 Stake Bed with Outrigger
4,Jennings,Leslie,1968 Ford Mustang
...,...,...,...
2991,Gerard,Martin,1954 Greyhound Scenicruiser
2992,Gerard,Martin,1950's Chicago Surface Lines Streetcar
2993,Gerard,Martin,Diamond T620 Semi-Skirted Tanker
2994,Gerard,Martin,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [15]:
# Your code here
q = """
SELECT lastName, firstName, SUM(quantityOrdered)
FROM employees as e
JOIN customers as c
ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
USING(customerNumber)
JOIN orderdetails
USING (orderNumber)
GROUP BY employeeNumber
ORDER BY lastName
"""
pd.read_sql(q, conn)



,lastName,firstName,SUM(quantityOrdered)
0,Bondur,Loui,6186
1,Bott,Larry,8205
2,Castillo,Pamela,9290
3,Firrelli,Julie,4227
4,Fixter,Andy,6246
5,Gerard,Martin,4180
6,Hernandez,Gerard,14231
7,Jennings,Leslie,11854
8,Jones,Barry,7486
9,Marsh,Peter,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [18]:
# Your code here
q = """
SELECT lastName, firstName, COUNT(productCode) AS different_product_sales
FROM employees AS e
JOIN customers AS c
ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
USING(customerNumber)
JOIN orderdetails
USING(orderNumber)
GROUP BY employeeNumber
HAVING different_product_sales > 200
ORDER BY different_product_sales DESC;
"""
pd.read_sql(q, conn)

,lastName,firstName,different_product_sales
0,Hernandez,Gerard,396
1,Jennings,Leslie,331
2,Castillo,Pamela,272
3,Bott,Larry,236
4,Jones,Barry,220
5,Vanauf,George,211


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!